In [1]:
#CONFIG INICIAL

import requests
import pandas as pd
import os, json, ast
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
HEADERS = {"accept": "application/json"}
RAW_DIR = "../dados/raw"
os.makedirs(RAW_DIR, exist_ok=True)


def get_json(url, params=None, retries=3):
    for tentativa in range(retries):
        try:
            response = requests.get(url, headers=HEADERS, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"[ERRO] Status {response.status_code} - {url}")
        except Exception as e:
            print(f"[EXCEÇÃO] {e}")
        return None

## 1. Deputados

In [2]:
# 1. Deputados
def get_deputados(legislatura):
    print(f"[INICIANDO] Coleta de deputados da legislatura {legislatura}...")
    url = f"{BASE_URL}/deputados"
    data = get_json(url, {"itens": 1000, "idLegislatura": legislatura})
    
    if data and "dados" in data:
        df = pd.DataFrame(data["dados"])
        path = os.path.join(RAW_DIR, f"deputados_legislatura_{legislatura}.csv")
        df.to_csv(path, index=False)
        print(f"[FINALIZADO] {len(df)} deputados salvos em '{path}'")
        return df
    else:
        print("[ERRO] Nenhum dado retornado.")
        return pd.DataFrame()

In [3]:
df_deputados = get_deputados(56)
df_deputados.head()

[INICIANDO] Coleta de deputados da legislatura 56...
[FINALIZADO] 1000 deputados salvos em '../dados/raw\deputados_legislatura_56.csv'


,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PHS,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
1,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PR,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
2,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PL,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
3,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PSC,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
4,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,None


## 2. Partidos

In [4]:
# 2. Partidos
def get_partidos():
    print("[Início] Partidos")
    data = get_json(f"{BASE_URL}/partidos")
    df = pd.DataFrame(data["dados"]).rename(columns={
        "id":   "id_partido",
        "sigla":"sigla_partido",
        "nome": "nome_partido"
    })
    df = df.drop(columns=[c for c in df.columns if "uri" in c.lower()], errors="ignore")
    path = os.path.join(RAW_DIR, "partidos.csv")
    df.to_csv(path, index=False)
    print(f"[OK] {len(df)} partidos salvos")
    return df

In [5]:
df_partidos = get_partidos()
df_partidos.head()

[Início] Partidos
[OK] 15 partidos salvos


,id_partido,sigla_partido,nome_partido
0,36898,AVANTE,Avante
1,37905,CIDADANIA,Cidadania
2,36899,MDB,Movimento Democrático Brasileiro
3,37901,NOVO,Partido Novo
4,36779,PCdoB,Partido Comunista do Brasil


## 3. Proposições

In [6]:
# 3. Proposições
def get_proposicoes_por_ano(ano):
    print(f"[Início] Proposições {ano}")
    todas, pag = [], 1
    while True:
        data = get_json(f"{BASE_URL}/proposicoes", {"ano": ano, "pagina": pag, "itens": 1000})
        if not data or not data["dados"]:
            break
        todas.extend(data["dados"]);  pag += 1
    df = pd.DataFrame(todas).rename(columns={
        "id": "id_proposicao",
        "siglaTipo": "sigla_tipo",
        "codTipo": "cod_tipo"
    })
    df = df.drop(columns=[c for c in df.columns if "uri" in c.lower()], errors="ignore")
    path = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    df.to_csv(path, index=False)
    print(f"[OK] {len(df)} proposições salvas")
    return df

In [7]:
for ano in range(2019,2023):
    df_proposicoes = get_proposicoes_por_ano(ano)
df_proposicoes.head()


[Início] Proposições 2019


[OK] 25728 proposições salvas
[Início] Proposições 2020
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes
[OK] 7800 proposições salvas
[Início] Proposições 2021
[OK] 21712 proposições salvas
[Início] Proposições 2022
[OK] 13760 proposições salvas


,id_proposicao,sigla_tipo,cod_tipo,numero,ano,ementa
0,303153,PL,139,618,2022,Dispõe sobre o exercício da profissão de Podól...
1,493361,PL,139,2253,2022,Dispõe sobre o monitoramento por instrumentos ...
2,587207,PL,139,1367,2022,Dispõe sobre a prestação dos serviços de contr...
3,597587,PL,139,3062,2022,"Altera a redação dos arts. 14, 17 e 18 da Lei ..."
4,996806,PL,139,1637,2022,Dispõe sobre a avaliação psicológica de gestan...


AQUI NO MEIO SERIA INTERESSANTE ADICIONAR UM TRATAMENTO NO ARQUIVO proposicoes_2023, POIS A CÉLULA ABAIXO UTILIZA ESSE ARQUIVO COMO INPUT

## 4. Detalhes Proposições

In [8]:
# 4. Detalhes proposições

def get_votacoes_por_proposicoes_parallel(ano, max_workers=10):
    print(f"[Início] Votações → proposições {ano}")
    prop_path = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    props = pd.read_csv(prop_path)["id_proposicao"]
    
    def fetch(prop_id):
        url  = f"{BASE_URL}/proposicoes/{prop_id}/votacoes"
        dat  = get_json(url)
        if dat and "dados" in dat:
            for d in dat["dados"]:
                d["id_proposicao"] = prop_id
            return dat["dados"]
        return []
    
    votacoes = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(fetch, pid) for pid in props]
        for i, fut in enumerate(as_completed(futures), 1):
            votacoes.extend(fut.result())
            if i % 500 == 0:
                print(f"{i}/{len(props)} proposições processadas")
    
    df = pd.DataFrame(votacoes)
    if not df.empty:
        df = df.rename(columns={
            "id": "id_votacao",
            "siglaOrgao": "sigla_orgao",
            "dataHoraRegistro": "data_hora_registro"
        }).drop(columns=[c for c in df.columns if "uri" in c.lower()], errors="ignore")
    path = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    df.to_csv(path, index=False)
    print(f"[OK] {len(df)} votações salvas")
    return df

for ano in range(2019, 2023):
    get_votacoes_por_proposicoes_parallel(ano)



[Início] Votações → proposições 2019
500/25728 proposições processadas
1000/25728 proposições processadas
1500/25728 proposições processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes/2191911/votacoes
2000/25728 proposições processadas
2500/25728 proposições processadas
3000/25728 proposições processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes/2194537/votacoes
3500/25728 proposições processadas
4000/25728 proposições processadas
4500/25728 proposições processadas
5000/25728 proposições processadas
5500/25728 proposições processadas
6000/25728 proposições processadas
6500/25728 proposições processadas
7000/25728 proposições processadas
7500/25728 proposições processadas
8000/25728 proposições processadas
8500/25728 proposições processadas
9000/25728 proposições processadas
9500/25728 proposições processadas
10000/25728 proposições processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes/2204096

In [10]:
#for ano in range(2019, 2023):
#    get_votacoes_por_proposicoes_parallel(ano)
#df_votacoes.head()

## 5. Votos individuais

In [19]:
# 5. Votos individuais referentes as proposições
import ast

def get_votos_individuais_proposicoes(ano, max_workers=10):
    print(f"[Início] Votos individuais {ano}")
    vot_path = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    votacoes = pd.read_csv(vot_path)["id_votacao"].tolist()
    
    def fetch(vot_id):
        data = get_json(f"{BASE_URL}/votacoes/{vot_id}/votos")
        if data and "dados" in data:
            for d in data["dados"]:
                d["id_votacao"] = vot_id
            return data["dados"]
        return []
    
    votos = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(fetch, vid) for vid in votacoes]
        for i, fut in enumerate(as_completed(futures), 1):
            votos.extend(fut.result())
            if i % 250 == 0:
                print(f"{i}/{len(votacoes)} votações processadas")

    df = pd.DataFrame(votos)

    if not df.empty and "deputado" in df.columns:
        # Remove registros onde "deputado" está vazio ou inválido
        df = df[df["deputado"].notna()]
        df = df[df["deputado"].astype(str).str.startswith("{")]

        # Converte votos para binário
        df["tipoVoto"] = df["tipoVoto"].map({"Sim": 1, "Não": 0}).fillna(-1)

        # Expande o dicionário de deputado
        def parse_dep(dep):
            try:
                return ast.literal_eval(dep)
            except:
                return {}

        dep_expand = df["deputado"].apply(parse_dep).apply(pd.Series)
        df = pd.concat([df.drop(columns=["deputado"]), dep_expand[["id"]]], axis=1)
        df = df.rename(columns={"id": "id_deputado"}).drop_duplicates()

        df = df[["id_votacao", "id_deputado", "tipoVoto"]]
    else:
        print(f"[AVISO] Nenhum voto com deputado encontrado em {ano}")
        df = pd.DataFrame(columns=["id_votacao", "id_deputado", "tipoVoto"])

    path_out = os.path.join(RAW_DIR, f"votos_individuais_proposicoes_{ano}.csv")
    df.to_csv(path_out, index=False)
    print(f"[OK] {len(df)} votos salvos")
    return df



In [20]:

for ano in range(2019, 2023):
    get_votos_individuais_proposicoes(ano)
#df_votos.head()

[Início] Votos individuais 2019
250/11450 votações processadas
500/11450 votações processadas
750/11450 votações processadas
1000/11450 votações processadas
1250/11450 votações processadas
1500/11450 votações processadas
1750/11450 votações processadas
2000/11450 votações processadas
2250/11450 votações processadas
2500/11450 votações processadas
2750/11450 votações processadas
3000/11450 votações processadas
3250/11450 votações processadas
3500/11450 votações processadas
3750/11450 votações processadas
4000/11450 votações processadas
4250/11450 votações processadas
4500/11450 votações processadas
4750/11450 votações processadas
5000/11450 votações processadas
5250/11450 votações processadas
5500/11450 votações processadas
5750/11450 votações processadas
6000/11450 votações processadas
6250/11450 votações processadas
6500/11450 votações processadas
6750/11450 votações processadas
7000/11450 votações processadas
7250/11450 votações processadas
7500/11450 votações processadas
7750/11450 

## 6. Orientações partidárias

In [21]:
# 6. Orientações partidárias por votação
def get_orientacoes_por_proposicoes(ano, max_workers=10):
    print(f"[Início] Orientações partidárias {ano}")
    vot_path = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    votacoes  = pd.read_csv(vot_path)["id_votacao"].tolist()
    
    def fetch(vot_id):
        data = get_json(f"{BASE_URL}/votacoes/{vot_id}/orientacoes")
        if data and "dados" in data:
            for d in data["dados"]:
                d["id_votacao"] = vot_id
            return data["dados"]
        return []
    
    orients = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(fetch, vid) for vid in votacoes]
        for i, fut in enumerate(as_completed(futures), 1):
            orients.extend(fut.result())
            if i % 250 == 0:
                print(f"{i}/{len(votacoes)} votações processadas")
    
    df = pd.DataFrame(orients).rename(columns={
        "orientacaoVoto": "orientacao",
        "siglaPartidoBloco": "sigla_partido"
    })
    df = df[["id_votacao", "sigla_partido", "orientacao"]]
    df.to_csv(os.path.join(RAW_DIR, f"orientacoes_proposicoes_{ano}.csv"), index=False)
    print(f"[OK] {len(df)} orientações salvas")
    return df


In [22]:

for ano in range(2019, 2023):
    get_orientacoes_por_proposicoes(ano)

#df_orientacoes.head()


[Início] Orientações partidárias 2019
250/11450 votações processadas
500/11450 votações processadas
750/11450 votações processadas
1000/11450 votações processadas
1250/11450 votações processadas
1500/11450 votações processadas
1750/11450 votações processadas
2000/11450 votações processadas
2250/11450 votações processadas
2500/11450 votações processadas
2750/11450 votações processadas
3000/11450 votações processadas
3250/11450 votações processadas
3500/11450 votações processadas
3750/11450 votações processadas
4000/11450 votações processadas
4250/11450 votações processadas
4500/11450 votações processadas
4750/11450 votações processadas
5000/11450 votações processadas
5250/11450 votações processadas
5500/11450 votações processadas
5750/11450 votações processadas
6000/11450 votações processadas
6250/11450 votações processadas
6500/11450 votações processadas
6750/11450 votações processadas
7000/11450 votações processadas
7250/11450 votações processadas
7500/11450 votações processadas
7750/

KeyboardInterrupt: 

## 7. Autores/temas das proposições

In [ ]:
def fetch_autores(prop_id):
    d = get_json(f"{BASE_URL}/proposicoes/{prop_id}/autores")
    return [{"id_proposicao": prop_id, **a} for a in d.get("dados", [])] if d else []

def fetch_temas(prop_id):
    d = get_json(f"{BASE_URL}/proposicoes/{prop_id}/temas")
    return [{"id_proposicao": prop_id, **t} for t in d.get("dados", [])] if d else []

def coleta_autores_temas(ano, max_workers=10):
    print(f"[Início] Autores & Temas {ano}")
    props = pd.read_csv(os.path.join(RAW_DIR, f"proposicoes_{ano}.csv"))["id_proposicao"]
    autores, temas = [], []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(fetch_autores, pid): "a" for pid in props}
        futs.update({ex.submit(fetch_temas, pid): "t" for pid in props})
        for i, (fut, tag) in enumerate(zip(futs, futs.values()), 1):
            res = fut.result()
            if tag == "a": autores.extend(res)
            else: temas.extend(res)
            if i % 500 == 0: print(f"{i//2}/{len(props)} proposições processadas")
    pd.DataFrame(autores).to_csv(os.path.join(RAW_DIR, f"autores_proposicoes_{ano}.csv"), index=False)
    pd.DataFrame(temas).to_csv(os.path.join(RAW_DIR, f"temas_proposicoes_{ano}.csv"), index=False)
    print("[OK] Autores & temas salvos")




In [ ]:
for ano in range(2019, 2023):
    coleta_autores_temas(ano)
#df_autores.head()

[INICIANDO] Coleta de autores das proposições do ano 2023...
[INFO] Proposição 100/22900 processada
[INFO] Proposição 200/22900 processada
[INFO] Proposição 300/22900 processada
[INFO] Proposição 400/22900 processada
[INFO] Proposição 500/22900 processada
[INFO] Proposição 600/22900 processada
[INFO] Proposição 700/22900 processada
[INFO] Proposição 800/22900 processada
[INFO] Proposição 900/22900 processada
[INFO] Proposição 1000/22900 processada
[INFO] Proposição 1100/22900 processada
[INFO] Proposição 1200/22900 processada
[INFO] Proposição 1300/22900 processada
[INFO] Proposição 1400/22900 processada
[INFO] Proposição 1500/22900 processada
[INFO] Proposição 1600/22900 processada
[INFO] Proposição 1700/22900 processada
[INFO] Proposição 1800/22900 processada
[INFO] Proposição 1900/22900 processada
[INFO] Proposição 2000/22900 processada
[INFO] Proposição 2100/22900 processada
[INFO] Proposição 2200/22900 processada
[INFO] Proposição 2300/22900 processada
[INFO] Proposição 2400/22900

,uri,nome,codTipo,tipo,ordemAssinatura,proponente,id_proposicao
0,https://dadosabertos.camara.leg.br/api/v2/depu...,Giovani Cherini,10000,Deputado(a),1,1,618609
1,https://dadosabertos.camara.leg.br/api/v2/orga...,Poder Executivo,30000,Órgão do Poder Executivo,1,1,2345493
2,https://dadosabertos.camara.leg.br/api/v2/depu...,Mara Gabrilli,10000,Deputado(a),1,1,2074843
3,https://dadosabertos.camara.leg.br/api/v2/depu...,Efraim Filho,10000,Deputado(a),1,1,1198010
4,https://dadosabertos.camara.leg.br/api/v2/orga...,Poder Executivo,30000,Órgão do Poder Executivo,1,1,2345487
